## ***Face_Recognition***

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import os

# Define class names
classes = {
    0: "Kunal",
    1: "Priti",
    2: "Rohit",
    3: "Walter",
    4: "Harvey"
}

In [ ]:
# Function to load images and labels for training
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = tf.keras.preprocessing.image.load_img(os.path.join(folder, filename), target_size=(100, 100))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images.append(img_array)
        labels.append(int(folder[-1]) - 1)  # Extract the person number from the folder name and convert to 0-based index
    return images, labels

In [ ]:
# Load training data
train_data = []
train_labels = []
for i in range(1, 6):
    folder_name = f"person_{i}"
    images, labels = load_images_from_folder(folder_name)
    train_data.extend(images)
    train_labels.extend(labels)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

In [ ]:
# Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
val_images = val_images / 255.0

In [ ]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')  # 5 output classes for 5 people
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

In [ ]:
# Load test data
test_data = []
test_labels = []
for i in range(1, 6):
    folder_name = f"test_person_{i}"
    images, labels = load_images_from_folder(folder_name)
    test_data.extend(images)
    test_labels.extend(labels)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [ ]:
# Normalize test data
test_data = test_data / 255.0

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_acc)

In [ ]:
# Single prediction
def predict_single_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(100, 100))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    return f"Predicted person: {classes[predicted_class]}"

# Example of single prediction
image_path = "Kunal1.jpg"  # Replace with your image path
prediction_result = predict_single_image(image_path)
print(prediction_result)